In [2]:
%matplotlib inline
%config InlineBackend.figure_format='retina'

To install prerequesites:

```python
python -m pip install -r requriements.txt
```

In [5]:
from pylab import *

import astropy.cosmology as cosmo
from astropy.cosmology import Planck18
import astropy.units as u
import seaborn as sns

sns.set_context('notebook')
sns.set_style('ticks')
sns.set_palette('colorblind')

The yearly merger rate from a source whose merger rate in the source frame tracks the [Madau & Dickinson (2014)](https://ui.adsabs.harvard.edu/abs/2014ARA%26A..52..415M/abstract) star formation rate and has an instantaneous merger rate $R_{0.2}$ at $z = 0.2$ in the source frame (the GWTC-3 population paper, [Abbott, et al. (2022)](https://ui.adsabs.harvard.edu/abs/2021arXiv211103634T/abstract), quotes merger rate at $z=0.2$ as this is approximately the best-measured redshift) is
$$
\frac{\mathrm{d} N}{\mathrm{d} t_\mathrm{detector}} = R_{0.2} \alpha \int \mathrm{d} z \, \frac{1}{1+z} \frac{\mathrm{d} V}{\mathrm{d} z} \frac{\left( 1 + z \right)^{2.7}}{1 + \left( \frac{1+z}{1+1.9} \right)^{5.6}}
$$
where 
$$
\alpha \equiv \left[ \frac{\left( 1 + 0.2 \right)^{2.7}}{1 + \left( \frac{1+0.2}{1+1.9} \right)^{5.6}} \right]^{-1}
$$

In [22]:
def md_sfr(z):
    return (1+z)**2.7 / (1 + ((1+z)/(1+1.9))**5.6)
alpha = 1/md_sfr(0.2)
alpha0 = 1/md_sfr(0)

Performing the integral, we obtain:

In [7]:
zs = expm1(arange(start=log(1), stop=log(1+100), step=0.01))
I = alpha*trapz(4*pi/(1+zs)*Planck18.differential_comoving_volume(zs).to(u.Gpc**3/u.sr).value*md_sfr(zs), zs)
print('I = {:.1f} Gpc**3'.format(I))

I = 2461.3 Gpc**3


In other words,
$$
\frac{\mathrm{d} N}{\mathrm{d} t} = \frac{2500}{\mathrm{yr}} \left( \frac{R_{0.2}}{\mathrm{Gpc}^{-3} \, \mathrm{yr}^{-1}} \right) = \frac{6.7}{\mathrm{d}} \left( \frac{R_{0.2}}{\mathrm{Gpc}^{-3} \, \mathrm{yr}^{-1}} \right) = \frac{0.28}{\mathrm{hr}} \left( \frac{R_{0.2}}{\mathrm{Gpc}^{-3} \, \mathrm{yr}^{-1}} \right)
$$

[Abbott, et al. (2022)](https://ui.adsabs.harvard.edu/abs/2021arXiv211103634T/abstract) quotes a BBH merger rate of $R_{0.2} \in [18,44] \, \mathrm{Gpc}^{-3} \, \mathrm{yr}^{-1}$, or 

In [21]:
Rlow = 18
Rhigh = 44
print('{:.0f}-{:.0f} / yr, {:.0f}-{:.0f} / d, {:.1f}-{:.0f} / hr'.format(round(I*Rlow,-3), round(I*Rhigh, -4), round(I*Rlow/365.25, -1), round(I*Rhigh/365.25,-1), I*Rlow/(365.25*24), I*Rhigh/(365.25*24)))

44000-110000 / yr, 120-300 / d, 5.1-12 / hr


The BNS merger rate is quoted at $z = 0$ between $10$ and $1700 \, \mathrm{Gpc}^{-3} \, \mathrm{yr}^{-1}$; the median rate is about $300 \, \mathrm{Gpc}^{-3} \, \mathrm{yr}$, or 

In [26]:
Rbns = 300
print('{:f} / yr, {:f} / d , {:f} / hr'.format(round(Rbns*I*alpha0/alpha,-5), round(Rbns*I*alpha0/alpha/365.25,-2), round(Rbns*I*alpha0/alpha/365.25/24.0,-1)))

1200000.000000 / yr, 3300.000000 / d , 140.000000 / hr
